# GZLB
## UE6: Anisotropes Werkstoffverhalten, Verallgemeinertes Hooke'sches Gesetz, Festigkeitshypothesen



### Aufgabe 6.1: Spannungsanalyse (ESZ)

gesucht für den Spannunsgtensor 

$$\underline{\underline{\sigma}}=\begin{bmatrix}130 & -40\\-40 & 70 \end{bmatrix} \text{MPa}$$

die Hautpnormalspannungen $\sigma_I$ und $\sigma_{II}$ (Eigenwerte) sowie ihre korrespondierenden Richtungen $\underline{v}_I$ und $\underline{v}_{II}$ mit $\underline{v}=(v_x,v_y)^\intercal$

#### Schritt 1
importieren des Packages Numpy

In [1]:
import numpy as np

#### Schritt 2
erstellen des Spannungstensors $\underline{\underline{\sigma}}$ als numpy-array mit dem Namen `stress`

In [2]:
stress = np.array([[130,-40 ], [0-40,70]])

print("Werte des array stress:\n",stress)
print("Dimensionen des array stress:", stress.shape)

Werte des array stress:
 [[130 -40]
 [-40  70]]
Dimensionen des array stress: (2, 2)


#### Berechnung der Eigenwerte und Eigenvektoren

das Eigenwertproblem ist wie folgt definiert

$$ \left( \underline{\underline{\sigma}}-\lambda \underline{\underline{E}}\right)\underline{v}=0$$

$$ \det \begin{vmatrix}\left( \underline{\underline{\sigma}}-\lambda \underline{\underline{E}}\right)\end{vmatrix}=0 \qquad \forall \underline{v}\neq0$$

in der Library `numpy.linagl` finden Sie die Funktion `eig(a)` zur Lösung des Eigenwertproblems $\lambda_i$ und zur Berechnung der rechten Eigenvektoren $\underline{v}_i$

In [3]:
w ,v = np.linalg.eig(stress)

In [4]:
print("Erster Eigenwert %.2f und zweiter Eigenwert %.2f" %(*w,))
print("Mit den Eigenvektore:\n", v)

Erster Eigenwert 150.00 und zweiter Eigenwert 50.00
Mit den Eigenvektore:
 [[ 0.89442719  0.4472136 ]
 [-0.4472136   0.89442719]]


### Berechnung des Winkels zwischen zwei Vektoren

Für zwei Vektoren $\underline{v}_1,\underline{v}_2 \in \mathbb{R}^n$ kann der (nicht-orientierte) Winkel $\alpha$ über das Skalarprodukt berechnet werden als

$$
\alpha = \arccos \left( \frac{\underline{v}_1\cdot\underline{v}_2}{|\underline{v}_1|\,|\underline{v}_2|} \right),
\qquad \alpha \in [0,\pi].
$$

Diese Darstellung liefert immer den **kleineren Winkel ohne Vorzeichen**, d.\,h. es wird keine Information über die Drehrichtung (Uhrzeigersinn oder Gegenuhrzeigersinn) geliefert. In numerischen Implementierungen wird das Argument der $\arccos$-Funktion häufig auf den Bereich $[-1,1]$ beschränkt, um Rundungsfehler zu vermeiden.

---

#### Skalarprodukt

$$
\underline{v}_1\cdot\underline{v}_2
=
\begin{pmatrix}
v_{1,1}\\ \vdots \\ v_{1,n}
\end{pmatrix}
\cdot
\begin{pmatrix}
v_{2,1}\\ \vdots \\ v_{2,n}
\end{pmatrix}
=
\sum_{i=1}^n v_{1,i}\,v_{2,i}.
$$

---

#### Betrag (euklidische Norm - L2-Norm)

$$
|\underline{v}|
=
\|\underline{v}\|_2
=
\sqrt{\sum_{i=1}^n (v_i)^2}.
$$

---

## Orientierter Winkel mit $\mathrm{atan2}$ (2D)

In zwei Dimensionen ($\mathbb{R}^2$) ist häufig ein **vorzeichenbehafteter, orientierter Winkel** $\varphi$ von Interesse. Dieser berücksichtigt die Drehrichtung und liegt im Bereich

$$
\varphi \in (-\pi,\pi].
$$

Er kann robust mit der $\mathrm{atan2}$-Funktion berechnet werden:

$$
\varphi
=
\mathrm{atan2}\!\left(
\det(\underline{v}_1,\underline{v}_2),\;
\underline{v}_1\cdot\underline{v}_2
\right).
$$

Dabei ist die Determinante (2D-Kreuzprodukt)

$$
\det(\underline{v}_1,\underline{v}_2)
=
\begin{vmatrix}
v_{1,x} & v_{1,y} \\
v_{2,x} & v_{2,y}
\end{vmatrix}
=
v_{1,x}v_{2,y}-v_{1,y}v_{2,x},
$$

und das Skalarprodukt

$$
\underline{v}_1\cdot\underline{v}_2
=
v_{1,x}v_{2,x}+v_{1,y}v_{2,y}.
$$

---

#### Interpretation

- $\det(\underline{v}_1,\underline{v}_2) > 0$: Drehung gegen den Uhrzeigersinn $\Rightarrow \varphi > 0$  
- $\det(\underline{v}_1,\underline{v}_2) < 0$: Drehung im Uhrzeigersinn $\Rightarrow \varphi < 0$

Im Gegensatz zur $\arccos$-Formulierung liefert $\mathrm{atan2}$ somit neben dem Betrag auch die **Orientierung des Winkels**.

---

#### Zusammenhang

- $\arccos$: nicht-orientierter Winkel, $[0,\pi]$  
- $\mathrm{atan2}$: orientierter Winkel, $(-\pi,\pi]$

Für 2D gilt: Der Betrag von $\varphi$ entspricht dem mit $\arccos$ berechneten Winkel, $\mathrm{atan2}$ enthält jedoch zusätzlich die Drehrichtung.

In [21]:
def angle_rad(v1, v2):
    """ 
        Returns the angle in radians between vectors 'v1' and 'v2'::
        using the scalar product
    """
    v1_u = unit_vector(v1)      ## normalized/unit vector
    v2_u = unit_vector(v2)      ## normalized/unit vector
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))


In [22]:
def signed_angle_rad(v, v_ref=np.array([1.0, 0.0])):
    v = np.asarray(v, dtype=float)
    v_ref = np.asarray(v_ref, dtype=float)

    v /= np.linalg.norm(v)
    v_ref /= np.linalg.norm(v_ref)

    # 2D "cross product" z-component == determinant
    det = v_ref[0]*v[1] - v_ref[1]*v[0]
    dot = np.dot(v_ref, v)
    return np.arctan2(det, dot)   # (-pi, pi]

In [23]:
def unit_vector(vector):
    """ 
        Returns the unit vector of the vector.  
    """
    return vector / np.linalg.norm(vector)

### Berechnung der Winkel $\alpha$ und $\beta$


$$\alpha = \angle \left( v_I,e_x \right)$$

$$\beta = \angle \left( v_I,e_y \right)$$

dabei sind $e_x= [1,0]^\intercal$ und $e_y = [0,1]^\intercal$

In [24]:
for i,  vector in enumerate(v.T):
    alpha = angle_rad(vector,np.array([1,0]))
    print("Der Winkel alpha für den %s Eigenvektor ist %.3f" %(i+1,np.rad2deg(alpha)))
    beta = angle_rad(vector,np.array([0,1]))
    print("Der Winkel beta für den %s Eigenvektor ist %.3f" %(i+1,np.rad2deg(beta)))


Der Winkel alpha für den 1 Eigenvektor ist 26.565
Der Winkel beta für den 1 Eigenvektor ist 116.565
Der Winkel alpha für den 2 Eigenvektor ist 63.435
Der Winkel beta für den 2 Eigenvektor ist 26.565


In [25]:
for i, vector in enumerate(v.T):
    alpha = signed_angle_rad(vector, np.array([1, 0]))
    beta  = signed_angle_rad(vector, np.array([0, 1]))
    print(f"alpha {i+1}: {np.degrees(alpha):.3f} deg")
    print(f"beta  {i+1}: {np.degrees(beta):.3f} deg")

alpha 1: -26.565 deg
beta  1: -116.565 deg
alpha 2: 63.435 deg
beta  2: -26.565 deg
